## 데이터 셋을 섞어야 한다.

* 용어
- clean : original CIFAR10
- adv : adversarial CIFAR10
- aug : augmented CIFAR10

* 방법
1. clean(BASELINE)
2. clean + adv -> clean : adv = [(90:10), (80:20), (70:30), (60:40)]
3. clean + aug -> clean : aug = [(90:10), (80:20), (70:30), (60:40)]
4. clean + adv + aug -> 미정

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchvision import datasets, transforms, utils
import torchvision.models as models

import os, random
from tqdm import tqdm
from PIL import Image
import numpy as np

from options.train_options import TrainOptions
from networks.LeNet import LeNet
from utils.Logger import Logger

## Load clean examples(original datasets)

### clean train set

In [27]:
clean_train_examples = []

clean_train_dataset = datasets.CIFAR10(
    root='./datasets/cifar10/',
    train=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((.5, .5, .5), (.5, .5, .5))
    ])
)

clean_train_loader = torch.utils.data.DataLoader(
    dataset=clean_train_dataset, batch_size=64, shuffle=True, num_workers=2
)

# clean dataset -> (data, label) in clean_train_loader :
# data, label이 같은 형태
for (data, label) in tqdm(clean_train_loader):
    clean_train_examples.append((data, label))
    # print(label)
    # print(data)
    # break

for (data, label) in clean_train_examples:
    print(data)
    print(label)
    break
# 슬라이싱 먼저 진행
# clean_train_examples = clean_train_examples[:int(len(clean_train_examples) * 0.8)]
# print(int(len(clean_train_examples) * 0.8))

100%|██████████| 782/782 [00:08<00:00, 92.54it/s] tensor([[[[-0.5451, -0.4824, -0.6863,  ..., -0.6000, -0.8824, -0.8824],
          [-0.7098, -0.6549, -0.7490,  ..., -0.4588, -0.7098, -0.7804],
          [-0.4902, -0.4510, -0.6235,  ..., -0.5059, -0.6784, -0.8431],
          ...,
          [-0.2078,  0.2000,  0.3804,  ...,  0.1608,  0.1843, -0.1922],
          [-0.2941, -0.1216,  0.1451,  ...,  0.1608, -0.1451, -0.1686],
          [-0.0275,  0.1765,  0.0196,  ...,  0.1451,  0.1137, -0.0431]],

         [[-0.2314, -0.1529, -0.4353,  ..., -0.3569, -0.6784, -0.6941],
          [-0.5529, -0.5059, -0.4667,  ..., -0.2000, -0.4510, -0.5765],
          [-0.4275, -0.3725, -0.3490,  ..., -0.2157, -0.4667, -0.6549],
          ...,
          [-0.2706, -0.1922, -0.1216,  ...,  0.1216,  0.2078, -0.2392],
          [-0.2863, -0.3647, -0.3569,  ...,  0.0196, -0.1686, -0.2549],
          [-0.1686, -0.0353, -0.3020,  ..., -0.1137, -0.0353, -0.1059]],

         [[-0.7804, -0.6549, -0.8039,  ..., -0.6000,

## Load adversarial examples
### get adversarial examples
- adv_examples : 적대적 예시 리스트 [(data, labels), ..., ]
  - data : 이미지가 ToTensor()에 의해 tensor화 된 것
  - labels : 이미지에 대한 ground truth, Tensor([라벨])

### adversary train set

In [25]:
class Adversary_Dataset(torch.utils.data.Dataset):
    def __init__(self, file_list, transform):
        self.file_list = file_list
        self.transform = transform

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        img_path = self.file_list[index]
        img = Image.open(img_path)
        img_transformed = self.transform(img)

        label = int(img_path.split('/')[-2])

        return img_transformed, label

In [29]:
tf = transforms.ToTensor()

ADV_TRAIN_DIR = './adversarial_examples/CIFAR10/training/'

# label
classes = os.listdir(ADV_TRAIN_DIR + '0.05/')

adv_train_examples = [] # [(data, label), ..., ]
adv_img_list = list()

for _class in classes:
    # print(_class, ' processing')
    # images
    train_images = os.listdir(ADV_TRAIN_DIR + '0.05/' + _class + '/')
    for image in tqdm(train_images):
        # data = Image.open(ADV_TRAIN_DIR + '0.05/' + _class + '/' + image)
        # data = tf(data)

        # label = torch.tensor(np.array([int(_class)]))
        # adv_train_examples.append( (data, label) )
        adv_img_list.append(ADV_TRAIN_DIR + '0.05/' + _class + '/' + image)

adv_train_dataset = Adversary_Dataset(adv_img_list, tf)
# for shuffling check
# print(adv_train_examples[0][1])
# print(adv_train_examples[49_999][1])

# load data loader
adv_train_loader = torch.utils.data.DataLoader(
    dataset=adv_train_dataset, batch_size=64, shuffle=True, num_workers=2
)

# for (data, label) in adv_train_loader:
#     print(data.shape)
#     print(label.shape)
#     break

for (data, label) in tqdm(adv_train_loader):
    adv_train_examples.append((data, label))

for (data, label) in adv_train_examples:
    print(data)
    print(label)
    break

100%|██████████| 782/782 [00:10<00:00, 75.93it/s]
tensor([[[[0.0000, 0.0039, 0.0039,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.1333, 0.1451, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.5020, 0.4000, 0.0157,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0471, 0.0275,  ..., 0.0000, 0.0000, 0.0000],
          [0.2157, 0.2235, 0.0510,  ..., 0.0000, 0.0000, 0.0000],
          [0.5922, 0.4863, 0.0863,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0784, 0.0784, 0.0784,  ..., 0.0078, 0.0078, 0.0078],
          [0.0745, 0.0706, 0.0667,  ..., 0.0078, 0.0078, 0

### shuffle the adversarial examples
- 라벨 별로 들어 있는 적대적 예시 배열을 랜덤하게 섞는다.

In [5]:
# for training
random.shuffle(adv_train_examples)
# adv_examples = adv_examples[s]

# for shuffling check
print(adv_train_examples[0][1])
print(adv_train_examples[49_999][1])

# 먼저 슬라이싱을 진행
adv_train_examples = adv_train_examples[:int(len(adv_train_examples) * 0.2)]

tensor([6])
tensor([8])


### create mixed datasets
- clean의 일부와 adversary의 일부를 +로 합친다.
    ```
    clean+adv = clean[:int(len(clean) * 0.8)] + adv[:(int)len(adv) * 0.2]
    shuffle clean + adv
    ```
- 섞은 것을 학습에 활용한다.

In [6]:

# mix training
# print(int(len(clean_train_examples) * 0.8))
# print(int(len(adv_train_examples) * 0.2))
mixed_train_dataset = clean_train_examples + adv_train_examples
# mixed_train_dataset = clean_train_examples[:int(len(clean_train_examples) * 0.8)] + adv_train_examples[:int(len(adv_train_examples) * 0.2)]
# random.shuffle(mixed_train_dataset) # mix it

### make mixed data loader

In [7]:
# training
mixed_train_loader = torch.utils.data.DataLoader(
    mixed_train_dataset, batch_size=64, shuffle=True, num_workers=2
)

# for (data, label) in mixed_train_loader:
#     print(label.to(DEVICE))
#     print(data[0].to(DEVICE))
#     break


## Train
- 새로 합성한 데이터 셋을 활용해서 학습 진행
- TODO :: data loader 두 개 섞기

In [ ]:
USE_CUDA = torch.cuda.is_available()

if __name__ == '__main__':

    DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')
    print('Training will be activated in', DEVICE)

    NETWORK = 'lenet'
    # DATASET = opt.dataset
    SAVE_DIR = './models/LeNet/'
    NUM_CLASSES = 10

    IS_PRETRAINED = False
    IS_TRANSFERED = False

    EPOCHS = 30
    BATCH_SIZE = 64
    LEARNING_RATE = 0.001
    MOMENTUM = 0.9

    CRITERION = 'crossentropy'
    OPTIMIZER = 'sgd'

    logger = Logger(model=NETWORK, dataset='adversary cifar10')
    hyper_parameter_infos = """\
    - is_pretrained : %s
    - is_transfered : %s
    - epochs : %s
    - batch_size : %s
    - learning_rate : %s
    - momentum : %s
    - criterion : %s
    - optimizer : %s\
    """ % (IS_PRETRAINED, IS_TRANSFERED, EPOCHS, BATCH_SIZE, LEARNING_RATE, MOMENTUM, CRITERION, OPTIMIZER)
    logs = ''

    # models : lenet5, resnet18, vgg16
    if NETWORK == 'lenet':
        model = LeNet(num_classes=NUM_CLASSES)
    if NETWORK == 'resnet18':
        model = models.resnet18(pretrained=IS_PRETRAINED,
                                num_classes=NUM_CLASSES)
    if NETWORK == 'vgg16':
        model = models.vgg16(pretrained=IS_PRETRAINED, num_classes=NUM_CLASSES)

    print(NETWORK + ' will be used')
    print(hyper_parameter_infos)
    model.to(DEVICE)

    if CRITERION == 'crossentropy':
        criterion = nn.CrossEntropyLoss()
    if CRITERION == 'mseloss':
        criterion = nn.MSELoss()

    if OPTIMIZER == 'sgd':
        optimizer = optim.SGD(model.parameters(),
                              lr=LEARNING_RATE, momentum=MOMENTUM)
    if OPTIMIZER == 'adam':
        optimizer = optim.Adam(
            model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)

    model.train()

    for epoch in range(EPOCHS):
        # print('%d epoch started' % epoch)
        running_loss = .0
        for _, (data, target) in enumerate(mixed_train_loader):
            data[0], target = data.to(DEVICE), target.to(DEVICE)
            optimizer.zero_grad()

            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print('%d epoch loss : %.3f' %
              (epoch + 1, running_loss / len(mixed_train_loader)))
        logs += '%d epoch loss : %.3f' % (epoch + 1,
                                          running_loss / len(mixed_train_loader)) + '\n'

    # use last epoch only
    torch.save(model.state_dict(), SAVE_DIR + NETWORK + '_adversary_cifar10' + '_net.pth')
    logger.create_txt(hyper_parameter_infos, logs)
